In [18]:
import numpy as np
import cv2,os
import matplotlib.pyplot as plt
from __future__ import division
from scipy.io import loadmat
import csv
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal
from numpy.linalg import inv
import math  
import pickle


# 1. Collect red barrel pixels in LAB color space from hand-labeled training image

In [19]:
barrelred=[]

for filename in os.listdir('Train_Set'):
    train_bgr= cv2.imread(os.path.join('Train_Set',filename))
    train=cv2.cvtColor(train_bgr,cv2.COLOR_BGR2LAB)
    train_handlabel=loadmat('barrel red/'+os.path.join(os.path.splitext(filename)[0]+"."+'mat'))
    train_handlabel = train_handlabel['roi']
    
    for i in range (len(train_handlabel)): ## red for barrel
            barrelred.append(train[train_handlabel[i,0],train_handlabel[i,1]])
            
barrelred=np.array(barrelred)



# 2. EM: parameter estimation of red barrel pixel Gaussian mixture model

In [20]:
num_cluster=3
pi=np.array([1/3,1/3,1/3])
miu=np.array([[50,160,150],[100,170,155],[150,180,160]])
sigma=np.array([[[100,50,50],[50,100,50],[50,50,100]],[[100,50,50],[50,100,50],[50,50,100]],[[100,50,50],[50,100,50],[50,50,100]]])


for loop in range(10):
    dimension=3
    p0=(2*math.pi)**dimension
    p1=[]
    covinverse=[]
    for j in range(num_cluster):
        p1.append(1/math.sqrt(p0*np.linalg.det(sigma[j])))
        covinverse.append(inv(sigma[j]))
    p1=np.array(p1)
    covinverse=np.array(covinverse)
    
    
    #E step    
    gamma=[]
    for j in range(num_cluster):
        gamma_k=[]
        for n in range (len(barrelred)):
                x=barrelred[n]
                p2=np.matmul(np.matmul((x-miu[j]).T,covinverse[j]),x-miu[j])
                p3=math.exp(-p2/2)
                p4=p1[j]*p3
                gamma_k.append(p4)
        gamma.append(gamma_k)

    #normalization

    for n in range (len(barrelred)):
        add=0
        for j in range(num_cluster):
            add=add+pi[j]*gamma[j][n]
        if add>pow(10,-200):
            p5=pi[0]*gamma[0][n]/add
            p6=pi[1]*gamma[1][n]/add
            p7=pi[2]*gamma[2][n]/add
        else:
            p5=0
            p6=0
            p7=0
        gamma[0][n]=p5
        gamma[1][n]=p6
        gamma[2][n]=p7

    #M step
    miu=[]
    for j in range (num_cluster):
        tempsum1=np.array([0,0,0])
        tempsum2=0
        for n in range (len(barrelred)):
                tempsum1=tempsum1+gamma[j][n]*barrelred[n]
                tempsum2+=gamma[j][n]

        miu.append(tempsum1/tempsum2)
    miu=np.array(miu)

    sigma=[]
    for j in range(num_cluster):
            tempsum1=np.array([[0,0,0],[0,0,0],[0,0,0]])
            tempsum2=0
            for n in range (len(barrelred)):
                     tempsum1=tempsum1+gamma[j][n]*np.matmul(np.array([barrelred[n]-miu[j]]).T,np.array([barrelred[n]-miu[j]]))
                     tempsum2+=gamma[j][n]
            sigma.append(tempsum1/tempsum2)
            
    pi=[]
    for j in range(num_cluster):
            tempsum=0
            for n in range (len(barrelred)):
                tempsum+=gamma[j][n]
            pi.append(tempsum/len(barrelred))
    add=np.sum(pi)
    for j in range(num_cluster):
        pi[j]=pi[j]/add
        

In [27]:
f = open('store.pckl','wb')
pickle.dump(pi, f)
pickle.dump(miu,f)
pickle.dump(sigma,f)
f.close()



# 3 Linear regression for distance estimation

In [22]:
#collect samples from training image
train_wh=[]
train_distance=[]
#train_distanceinverse=[]
train_ratio=[]
for filename in os.listdir('Train_Set'):
    train_bgr= cv2.imread(os.path.join('Train_Set',filename))
    train_handlabel=loadmat('barrel red/'+os.path.join(os.path.splitext(filename)[0]+"."+'mat'))
    train_handlabel = train_handlabel['roi']
    p0=[]
    for i in range(len(train_handlabel)):
        p1=[]
        p2=[]
        p2.append(train_handlabel[i][1])
        p2.append(train_handlabel[i][0])
        p1.append(p2)
        p0.append(p1)
        
    x,y,w,h = cv2.boundingRect(np.array(p0,dtype=np.float32))
    train_ratio.append(w/h)
    cc=[]
    cc.append(1/w)
    cc.append(1/h)
    train_wh.append(cc)
    train_distance.append(float(filename.split('.')[0]))
    


In [24]:
f = open('store.pckl', 'wb')
pickle.dump(train_wh, f)
pickle.dump(train_distance,f)
f.close()





[5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 6.0,
 7.0,
 7.0,
 7.0,
 7.0,
 8.0,
 8.0,
 8.0,
 8.0,
 9.0]